In [4]:
import pandas as pd 
import numpy as np 

from sklearn.metrics.pairwise import cosine_similarity
from tqdm.auto import tqdm

In [1]:
from sentence_transformers import SentenceTransformer

# Embedding Vector 추출에 활용할 모델(jhgan/ko-sbert-sts) 불러오기
model = SentenceTransformer('jhgan/ko-sbert-sts', use_auth_token=False)

c:\Users\minkyu\AppData\Local\anaconda3\envs\sd2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\minkyu\AppData\Local\anaconda3\envs\sd2\lib\site-packages\sentence_transformers\SentenceTransformer.py:195: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v4 of SentenceTransformers.
  warnings.warn(


In [2]:
data_path = "C:/Users/minkyu/Desktop/open/"
base_path = 'C:/Users/minkyu/Desktop/dacon accident prevention'

In [5]:
train = pd.read_csv(data_path+"train.csv" )
test = pd.read_csv(data_path+"test.csv" )

In [6]:
grouped = train.groupby("인적사고")

In [7]:
res = {}
jaccard_res = []

# 자카드 유사도 함수
def jaccard_similarity(text1, text2):
    """자카드 유사도 계산"""
    set1, set2 = set(text1.split()), set(text2.split())  # 단어 집합 생성
    intersection = len(set1.intersection(set2))  # 교집합 크기
    union = len(set1.union(set2))  # 합집합 크기
    return intersection / union if union != 0 else 0

for name, group in tqdm(grouped):
    plans = group["재발방지대책 및 향후조치계획"].tolist()
    n = len(plans)
    
    # 유사도 행렬 초기화
    similarity_matrix = np.zeros((n, n))
    
    # 이중 for문을 통해 자카드 유사도 계산
    for i in range(n):
        for j in range(n):
            similarity_matrix[i, j] = jaccard_similarity(plans[i], plans[j])
    
    # 각 텍스트별 평균 유사도 계산
    avg_similarity = similarity_matrix.mean(axis=1)
    
    # 평균 유사도가 가장 높은 인덱스 찾기
    best_idx = avg_similarity.argmax()
    
    # 선택된 텍스트의 유사도 결과 저장
    jaccard_res += similarity_matrix[best_idx].tolist()
    res[name] = plans[best_idx]


100%|██████████| 23/23 [03:04<00:00,  8.00s/it]


In [9]:
arr = jaccard_res

# 0.1 단위로 구간을 지정
bins = np.arange(0, 1.1, 0.1)  # 0.0 ~ 1.0을 0.1 간격으로 나눔

# 히스토그램 계산
hist, bin_edges = np.histogram(arr, bins=bins)

# 결과 출력
for i in range(len(hist)):
    print(f"Range {bin_edges[i]:.1f} - {bin_edges[i+1]:.1f}: {hist[i]}개")

Range 0.0 - 0.1: 10847개
Range 0.1 - 0.2: 6568개
Range 0.2 - 0.3: 3214개
Range 0.3 - 0.4: 1309개
Range 0.4 - 0.5: 638개
Range 0.5 - 0.6: 379개
Range 0.6 - 0.7: 205개
Range 0.7 - 0.8: 87개
Range 0.8 - 0.9: 60개
Range 0.9 - 1.0: 83개


In [ ]:
res